In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:80% !important;}
div.prompt {min-width:70px;}
div#toc-header{margin-top:150px;}
span.toc-item-num{display:none;}
div.CodeMirror {font-family:Consolas}
div.input {font-family:Consolas}
</style>
"""))

<b><font color='red' size='6'>ch09. 파일 i/o 프로그래밍</font></b>

# 1절. 파일에 데이터를 저장하고 불러오기

In [15]:
# w 또는 wt : (write)  기존의 파일이 있는 경우 덮어쓰기, 파일이 없는 경우 생성하여 쓰기
# - 쓰기 모드는 폴더가 없는 경우 예외 발생
# a 또는 at : (append) 기존의 파일이 있는 경우 (맨 뒤에) append, 파일이 없는 경우 생성하여 쓰기
# r 또는 rt : (read)   기존의 파일을 읽는 경우, 파일이 없는 경우 예외 발생
f = open('data/ch09_sample.txt', 'w')
print(f.writable())

True


In [16]:
f.write('Hello\nWorld\nHello\nPython\n')  # 한글을 write하면 CP949로 저장됨(windows)
f.close()

In [17]:
# with 구문을 사용하면 매번 close하는 코드가 불필요함
with open('data/ch09_sample.txt', 'wt') as f:
    print('쓰기 가능한지 여부 :', f.writable())
    f.write('Python is easy.\nPython is smart.\n')

쓰기 가능한지 여부 : True


In [18]:
with open('data/ch09_sample.txt', 'at') as f:
    print('프린트 함수로 파일 쓰기가 가능합니다-콘솔 출력')
    print('프린트 함수로 파일 쓰기가 가능합니다-파일 출력', file=f)

프린트 함수로 파일 쓰기가 가능합니다-콘솔 출력


- euc-kr : 한글 완성형 (믜X)
- cp949  : 확장된 한글 완성형 (믜O) - 엑셀에서 
- utf-8  : 한글 조합형 (초중종성 따로 조합해서 저장) - 주피터노트북에서

In [19]:
with open('data/ch09_sample.txt', 'w', encoding='utf-8') as f:
    f.write('= 여러 줄의 text를 쓰기 =\n')
    f.write('1방법 : \nHello\nWorld\n')
    textlist = ['홍길동, 30, 서울\n', '김길동, 20, 안양\n']
    f.write('2방법 : \n')
    for line in textlist:
        f.write(line)
    f.write('3방법 : \n')
    f.writelines(textlist)

In [22]:
# 파일을 한 줄 읽고 출력, 한 줄 읽고 출력, ... text로 return
with open('data/ch09_sample.txt', 'r', encoding='utf-8') as f:
    line = f.readline()  # 한 줄 읽기
    while line != '':
        print(line, end='')  # 파일에 \n이 포함되어서 출력 시 개행은 제거
        line = f.readline()

= 여러 줄의 text를 쓰기 =
1방법 : 
Hello
World
2방법 : 
홍길동, 30, 서울
김길동, 20, 안양
3방법 : 
홍길동, 30, 서울
김길동, 20, 안양


In [23]:
# 파일을 한 번에 여러 줄 읽어 list를 return
with open('data/ch09_sample.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()  # list로 return
    print(lines)

['= 여러 줄의 text를 쓰기 =\n', '1방법 : \n', 'Hello\n', 'World\n', '2방법 : \n', '홍길동, 30, 서울\n', '김길동, 20, 안양\n', '3방법 : \n', '홍길동, 30, 서울\n', '김길동, 20, 안양\n']


In [25]:
for line in lines:
    print(line, end='')

= 여러 줄의 text를 쓰기 =
1방법 : 
Hello
World
2방법 : 
홍길동, 30, 서울
김길동, 20, 안양
3방법 : 
홍길동, 30, 서울
김길동, 20, 안양


# 2절. 피클을 이용한 객체 저장 및 불러오기
## 2.1 형식이 있는 텍스트 데이터 불러오기

In [37]:
with open('data/ch09_member.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    # print(lines)
for line in lines:
    # print(line)
    data = line.strip().split(',')  # strip() : 앞뒤에 white space 제거 (trim())
    name = data[0]
    age = int(data[1].strip())  # ' 20'이므로 strip() 해주기
    email = data[2].strip()
    address = data[3].strip()
    print('이름 : {}, 나이 : {}, 메일 : {}, 주소 : {}'.format(name, age, email, address))

이름 : 홍길동, 나이 : 20, 메일 : kildong@hong.com, 주소 : 서울시 관악구
이름 : 홍길숙, 나이 : 25, 메일 : kilsuk1@hong.com, 주소 : 서울시 영등포구
이름 : 신길동, 나이 : 30, 메일 : shinkil@hong.com, 주소 : 서울시 동작구


In [38]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return '이름 : {}, 나이 : {}, 메일 : {}, 주소 : {}'.format(
                                                            self.name, self.age, self.email, self.address)
    def as_dict(self):  # 객체를 딕셔너리로 바꿔 return
        return {'name' : self.name,
                'age' : self.age,
                'email' : self.email,
                'address' : self.address}

In [42]:
# 생성자 test
user = Member('홍길동', 22, 'hong@hong.com', '서울')
# __str__ test
print(user)
# as_dict test
print(user.as_dict())

이름 : 홍길동, 나이 : 22, 메일 : hong@hong.com, 주소 : 서울
{'name': '홍길동', 'age': 22, 'email': 'hong@hong.com', 'address': '서울'}


In [43]:
# 파일의 내용을 member list나 딕셔너리 list로 읽기
user_list = []  # member list
user_dict = []  # 딕셔너리 list
with open('data/ch09_member.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
for line in lines:
    data = line.strip().split(',')
    name = data[0]
    age = int(data[1].strip())
    email = data[2].strip()
    address = data[3].strip()
    user = Member(name, age, email, address)
    user_list.append(user)
    user_dict.append(user.as_dict())

In [45]:
# user_list (Member 객체 list)
for user in user_list:
    print(user)

이름 : 홍길동, 나이 : 20, 메일 : kildong@hong.com, 주소 : 서울시 관악구
이름 : 홍길숙, 나이 : 25, 메일 : kilsuk1@hong.com, 주소 : 서울시 영등포구
이름 : 신길동, 나이 : 30, 메일 : shinkil@hong.com, 주소 : 서울시 동작구


In [46]:
# user_dict (Member 객체 딕셔너리 list)
for user in user_dict:
    print(user)

{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'}
{'name': '홍길숙', 'age': 25, 'email': 'kilsuk1@hong.com', 'address': '서울시 영등포구'}
{'name': '신길동', 'age': 30, 'email': 'shinkil@hong.com', 'address': '서울시 동작구'}


In [47]:
# user_dict (딕셔너리 list)
user_dict

[{'name': '홍길동', 'age': 20, 'email': 'kildong@hong.com', 'address': '서울시 관악구'},
 {'name': '홍길숙',
  'age': 25,
  'email': 'kilsuk1@hong.com',
  'address': '서울시 영등포구'},
 {'name': '신길동', 'age': 30, 'email': 'shinkil@hong.com', 'address': '서울시 동작구'}]

## 2.2 피클링
 - 바이너리를 사용해 저장할 때는 피클을 이용해야 함

In [55]:
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def __str__(self):
        return '이름 : {}, 나이 : {}, 메일 : {}, 주소 : {}'.format(
                                                            self.name, self.age, self.email, self.address)
    def __eq__(self, other):
        '''
        user1.__eq__(user2) 호출 시 user1의 속성변수들과 user2의 속성변수들이 모두 같은지 여부를 return
        '''
        if isinstance(other, Member):
            return self.name == other.name and self.age == other.age and \
                            self.email == other.email and self.address == other.address
        else:
            return False
    def as_dict(self):  # 객체를 딕셔너리로 바꿔 return
        return {'name' : self.name,
                'age' : self.age,
                'email' : self.email,
                'address' : self.address}
    
user1 = Member('홍길동', 20, 'h@h.com', '서울')
user2 = Member('홍길동', 20, 'h@h.com', '서울')    
print(user1.__eq__(user2))
print(user1 == user2)

True
True


In [56]:
user1 = Member('홍길동', 20, 'h@h.com', '서울')
user2 = Member('김길동', 25, 'k@h.com', '인천')
user3 = Member('신길동', 30, 's@h.com', '포천')
user_list = [user1, user2, user3]

In [57]:
for user in user_list:
    print(user)

이름 : 홍길동, 나이 : 20, 메일 : h@h.com, 주소 : 서울
이름 : 김길동, 나이 : 25, 메일 : k@h.com, 주소 : 인천
이름 : 신길동, 나이 : 30, 메일 : s@h.com, 주소 : 포천


In [58]:
# 피클링을 이용한 객체를 파일에 저장하기
import pickle
with open('data/ch09_member.data', 'wb') as f:  # write binary
    pickle.dump(user_list, f)

In [59]:
# 변수만 지운 것이므로 내용은 남아있음
del user1, user2, user3

In [60]:
# 피클링을 이용한 파일에서 객체 데이터 읽기
with open('data/ch09_member.data', 'rb') as f:
    load_user_list = pickle.load(f)  # 객체 리스트로 가져옴

In [61]:
for user in load_user_list:
    print(user)

이름 : 홍길동, 나이 : 20, 메일 : h@h.com, 주소 : 서울
이름 : 김길동, 나이 : 25, 메일 : k@h.com, 주소 : 인천
이름 : 신길동, 나이 : 30, 메일 : s@h.com, 주소 : 포천


In [63]:
# user_list와 load_user_list가 모두 같은지 여부 확인
result = []
for idx in range(len(user_list)):
    # print(idx)
    result.append(user_list[idx] == load_user_list[idx])
result

[True, True, True]

In [64]:
all(result)  # 모든 요소들이 True인지 여부를 return

True

# 3절. csv 형식 파일 읽기/쓰기
 - CSV 파일의 자료를 리스트 데이터로 읽기    (3.1과 3.2)
 - CSV 파일의 자료를 딕셔너리 데이터로 읽기  (3.3과 3.4)
 
## 3.1 reader

In [68]:
# utf-8로 인코딩된 csv파일 읽기
import csv
with open('data/ch09_member1.csv', 'r', encoding='UTF-8') as f:
    reader = csv.reader(f)
    # print(list(reader))
    for row in reader:
        print(row)

['홍길동', '20', 'kildong@hong.com', '서울시 관악구']
['김길동', '40', 'kimdong@hong.com', '서울시 영등포구']
['신길동', '30', 'sindong@hong.com', '서울시 동작구']


In [1]:
# print(list(reader))  # 변수에 할당해야지 읽어들일 수 있음

In [71]:
# CP949로 인코딩된 csv파일 read
with open('data/ch09_member1-cp.csv') as f:  # 'r', encoding='CP949' : 기본값으로 생략 가능
    reader = csv.reader(f)
    result = list(reader)
# print(result)  # list 안에 list
for row in result:
    print(' - '.join(row))

홍길동 - 20 - kildong@hong.com - 서울시 관악구
김길동 - 40 - kimdong@hong.com - 서울시 영등포구 - 팀장
신길동 - 30 - sindong@hong.com - 서울시 동작구


In [73]:
# ""(따옴표)가 없는 데이터는 numeric으로 처리
with open('data/ch09_member1.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(
        f, quotechar='"',
        quoting=csv.QUOTE_NONNUMERIC)  # "가 없는 것은 numeric으로 가져오도록
    for row in reader:
        print(row)  # 기본적으로 float로 가져옴

['홍길동', 20.0, 'kildong@hong.com', '서울시 관악구']
['김길동', 40.0, 'kimdong@hong.com', '서울시 영등포구']
['신길동', 30.0, 'sindong@hong.com', '서울시 동작구']


In [75]:
# csv 파일의 내용을 딕셔너리 리스트로 가져오기
user_dict = []
with open('data/ch09_member1.csv', encoding='utf-8') as f:
    reader = csv.reader(f, quotechar='"', quoting=csv.QUOTE_NONNUMERIC)
    for row in reader:
        user_dict.append({'name' : row[0], 'age' : row[1], 'email' : row[2], 'address' : row[3], })
print('파일에서 읽어온 딕셔너리 리스트')
print(user_dict)

파일에서 읽어온 딕셔너리 리스트
[{'name': '홍길동', 'age': 20.0, 'email': 'kildong@hong.com', 'address': '서울시 관악구'}, {'name': '김길동', 'age': 40.0, 'email': 'kimdong@hong.com', 'address': '서울시 영등포구'}, {'name': '신길동', 'age': 30.0, 'email': 'sindong@hong.com', 'address': '서울시 동작구'}]


## 3.2 writer

In [76]:
user_list = [['홍길동',20,'hong@hong.com','서울'],
             ['신길동',30,'shin@hong.com','인천'],]

In [77]:
# newline='' 옵션을 빼면 개행이 2번 이루어짐
try:
    with open('data/ch09_member2-write-cp.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        for user in user_list:
            writer.writerow(user)
except FileNotFoundError as e:
    print(e)

In [79]:
with open('data/ch09_member2-write.csv', 'w', encoding='utf-8', newline='') as f:  # w모드는 생략 불가능
    writer = csv.writer(f, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerows(user_list)

## 3.3 DictReader
 - csv파일의 자료를 딕셔너리 데이터로 읽기

In [82]:
# header가 있는 csv 파일을 딕셔너리로 읽기(ch09_member3.csv)
with open('data/ch09_member3.csv', 'r', encoding='utf-8') as f:
    dict_reader = csv.DictReader(f)
    # print(list(dict_reader))
    for row in dict_reader:
        if row['job'] is None:
            print(row['name'], row['age'], row['email'], row['address'])
        else:
            print(row['name'], row['age'], row['email'], row['address'], row['job'])

홍길동 20 h@h.com 서울시 관악구
신길동 40 s@s.com 서울시 영등포구 팀장
김길동 30 k@k.com 서울시 동작구


In [84]:
# open() encoding='CP949'가 디폴트
# cf. csv파일을 데이터프레임으로
import pandas as pd
member = pd.read_csv('data/ch09_member3.csv', encoding='utf-8')  # encoding='utf-8'이 기본
member

,name,age,email,address,job
0,홍길동,20,h@h.com,서울시 관악구,NaN
1,신길동,40,s@s.com,서울시 영등포구,팀장
2,김길동,30,k@k.com,서울시 동작구,NaN


In [85]:
type(member)

pandas.core.frame.DataFrame

In [87]:
# header가 없는 csv 파일을 딕셔너리로 읽기(ch09_member1-cp.csv)
with open('data/ch09_member1-cp.csv', 'r', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, fieldnames=['Name', 'Age', 'Email', 'Address'])
    for row in dict_reader:
        # print(row)
        # 지정한 header에 Job이 없으므로 무시되어 출력되지 않음
        print(row['Name'], row['Age'], row['Email'], row['Address'])

홍길동 20 kildong@hong.com 서울시 관악구
김길동 40 kimdong@hong.com 서울시 영등포구
신길동 30 sindong@hong.com 서울시 동작구


In [89]:
# header가 없는 csv 파일을 딕셔너리로 읽기(ch09_member1-cp.csv)
# Job까지 가져오기
with open('data/ch09_member1-cp.csv', 'r', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, fieldnames=['Name', 'Age', 'Email', 'Address', 'Job'])
    for row in dict_reader:
        # print(row)
        print(row['Name'], row['Age'], row['Email'], row['Address'], row['Job'])

{'Name': '홍길동', 'Age': '20', 'Email': 'kildong@hong.com', 'Address': '서울시 관악구', 'Job': ''}
홍길동 20 kildong@hong.com 서울시 관악구 
{'Name': '김길동', 'Age': '40', 'Email': 'kimdong@hong.com', 'Address': '서울시 영등포구', 'Job': '팀장'}
김길동 40 kimdong@hong.com 서울시 영등포구 팀장
{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}
신길동 30 sindong@hong.com 서울시 동작구 


In [91]:
# header가 없는 csv 파일을 딕셔너리로 읽기(ch09_member1-cp.csv) / Job까지 가져오기
with open('data/ch09_member1-cp.csv', 'r', encoding='cp949') as f:
    dict_reader = csv.DictReader(f, fieldnames=['Name', 'Age', 'Email', 'Address'], restkey='Job')
    for row in dict_reader:
        # print(row)
        if row['Job'][0] == '':
            print(row['Name'], row['Age'], row['Email'], row['Address'])
        else:
            print(row['Name'], row['Age'], row['Email'], row['Address'], row['Job'][0])

홍길동 20 kildong@hong.com 서울시 관악구
김길동 40 kimdong@hong.com 서울시 영등포구 팀장
신길동 30 sindong@hong.com 서울시 동작구


In [101]:
# ch09_member1.cp.csv의 내용을 딕셔너리 list로 받아보세요
userlist = []  # 딕셔너리 list
# 파일의 내용을 userlist에 담ㄱ
with open('data/ch09_member1-cp.csv', 'r', encoding='CP949') as f:
          dict_reader = csv.DictReader(f, fieldnames=['Name', 'Age', 'Email', 'Address', 'Job'])
          for row in dict_reader:
                userlist.append(row)
print(row)                
userlist

{'Name': '신길동', 'Age': '30', 'Email': 'sindong@hong.com', 'Address': '서울시 동작구', 'Job': ''}


[{'Name': '홍길동',
  'Age': '20',
  'Email': 'kildong@hong.com',
  'Address': '서울시 관악구',
  'Job': ''},
 {'Name': '김길동',
  'Age': '40',
  'Email': 'kimdong@hong.com',
  'Address': '서울시 영등포구',
  'Job': '팀장'},
 {'Name': '신길동',
  'Age': '30',
  'Email': 'sindong@hong.com',
  'Address': '서울시 동작구',
  'Job': ''}]

## 3.4 DictWriter
 - 딕셔너리 리스트를 csv파일로 쓰기

In [115]:
user1 = {'name' : '홍길동', 'age' : 20, 'email' : 'h@hong.com', 'address' : '서울'}
user2 = {'name' : '신길동', 'age' : 25, 'email' : 's@hong.com', 'address' : '부산'}
user3 = {'name' : '김길동', 'age' : 35, 'email' : 'k@hong.com', 'address' : '인천'}
user_list = [user1, user2, user3]
fieldnames = list(user1.keys())
print(fieldnames)

['name', 'age', 'email', 'address']


```
csv 파일 내용
naem,age,email,address
홍길동,20,h@hong.com,서울
...
```

In [119]:
with open('data/ch09_member4.csv', 'w', encoding='UTF-8', newline='') as f:
    dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
    dict_writer.writeheader()  # header 한 줄 쓰기
    # for user in user_list:
    #     dict_writer.writerow(user)
    dict_writer.writerows(user_list)

# 4절. JSON 파일에 저장하고 불러오기(dump, load)
[{'name' : '홍,
  'age' : 20},
 {'name' : '박,
  'age' : 30}]
## 4.1 JSON dump(파일에 저장)
1. 딕셔너리 리스트 데이터를 json파일로
2. 객체 리스트 데이터를 json파일로

In [4]:
data = [{'name' : '홍길동', 'age' : 20, 'email' : 'h@h.com', 'address' : '서울'},
       {'name' : '김길동', 'age' : 30, 'email' : 'k@h.com', 'address' : '부산'},
       {'name' : '신길동', 'age' : 40, 'email' : 's@h.com', 'address' : '안양'}]

In [7]:
# 1. 딕셔너리 리스트 데이터를 json파일로 저장(dump)
# ensure_ascii옵션의 역할 : ASCII 코드 범위를 넘어서는 문자는 어떻게 처리할지 여부
# True(기본값) : ASCII 코드 범위를 넘어선 한글은 유니코드로 저장
# False : 비 ASCII 문자 원래 형태 그대로 저장
# indent=' ' : 속성들마다 ' ' 하나로 들여쓰기
import json
with open('data/ch09_member1.json', 'w') as jsonfile:  # encoding='cp949' 기본값
    json.dump(data,   # 딕셔너리 리스트
              jsonfile, ensure_ascii=False, indent=' ')

In [8]:
# 2. 객체 리스트 데이터를 json파일로 저장(dump)
class Member:
    def __init__(self, name, age, email, address):
        self.name = name
        self.age = age
        self.email = email
        self.address = address
    def as_dict(self):
        '객체를 딕셔너리로 바꿔 return (json.dump 시 필요)'
        return {'name' : self.name, 'age' : self.age, 'email' : self.email, 'address' : self.address}
    def __str__(self):
        return '이름 : {}, 나이 : {}살, 메일 : {}, 주소 : {}'.format(
                                                            self.name, self.age, self.email, self.address)
    def __eq__(self, other):
        '''
        m1.__eq__(m2) 호출 시 두 객체의 속성 변수들의 값이 모두 일치하는지 여부
        m1 == m2 구현 시 두 객체의 속성 변수들의 값이 모두 일치하는지 여부
        '''
        if isinstance(other, Member):  # other가 Member인지
            return self.name == other.name and \
                    self.age == other.age and \
                    self.email == other.email and \
                    self.address == other.address
        else:
            return False

In [11]:
member_list = [Member('홍길동', 30, 'h@h.com', '서울'),
               Member('김길동', 25, 'k@h.com', '부산'),
               Member('신길동', 35, 's@h.com', '대전'), ]

In [12]:
for member in member_list:
    print(member)

이름 : 홍길동, 나이 : 30살, 메일 : h@h.com, 주소 : 서울
이름 : 김길동, 나이 : 25살, 메일 : k@h.com, 주소 : 부산
이름 : 신길동, 나이 : 35살, 메일 : s@h.com, 주소 : 대전


In [13]:
with open('data/ch09_member2.json', 'a', encoding='UTF8') as jsonfile:
    json.dump(member_list,  # 객체 리스트
              jsonfile, ensure_ascii=False, indent='\t',
              default=Member.as_dict)  # 객체를 딕셔너리 형태로 return하는 인스턴스 함수

In [14]:
# 현재 세션(현재 실행 중인 jupyter notebook)에 존재하는 변수들의 목록
# %~ 매직명령어 : jupyter notebook의 IPYTHON 환경에서만 제공되는 명령어
%whos

Variable        Type             Data/Info
------------------------------------------
HTML            type             <class 'IPython.core.display.HTML'>
Member          type             <class '__main__.Member'>
autopep8        module           <module 'autopep8' from '<...>e-packages\\autopep8.py'>
data            list             n=3
display         function         <function display at 0x000001B796E27D90>
json            module           <module 'json' from 'C:\\<...>\lib\\json\\__init__.py'>
jsonfile        TextIOWrapper    <_io.TextIOWrapper name='<...>mode='a' encoding='UTF8'>
member          Member           이름 : 신길동, 나이 : 35살, 메일 : s@h.com, 주소 : 대전
member_list     list             n=3
yapf_reformat   function         <function yapf_reformat at 0x000001B798C03E20>


## 4.2 JSON load(파일 읽기)
1. json파일을 딕셔너리 리스트 변수로 읽기
2. json파일을 객체 리스트 변수로 읽기

In [15]:
# 1. json파일을 딕셔너리 리스트로
with open('data/ch09_member1.json', 'r', encoding='cp949') as jsonfile:
    load_data = json.load(jsonfile)
for row in load_data:
    print(row)

{'name': '홍길동', 'age': 20, 'email': 'h@h.com', 'address': '서울'}
{'name': '김길동', 'age': 30, 'email': 'k@h.com', 'address': '부산'}
{'name': '신길동', 'age': 40, 'email': 's@h.com', 'address': '안양'}


In [17]:
type(load_data), type(load_data[0])

(list, dict)

In [18]:
# 2. json파일(ch09_member2.json)을 객체 리스트로

In [19]:
def as_member(dic):
    '딕셔너리 데이터를 매개변수로 받아 객체로 return하는 함수'
    return Member(dic['name'], dic['age'], dic['email'], dic['address'])

In [20]:
with open('data/ch09_member2.json', 'r', encoding='UTF-8') as jsonfile:
    load_member_list = json.load(jsonfile, object_hook=as_member)

In [21]:
type(load_member_list), type(load_member_list[0])

(list, __main__.Member)

In [22]:
# member_list => ch09_member2.json => load_member_list로 받음

In [27]:
# member_list와 load_member_list의 모든 데이터가 일치하는지 여부를 확인
result = []
for idx in range(len(member_list)):
    # result.append(member_list[idx].__eq__(load_member_list[idx]))
    result.append(member_list[idx] == load_member_list[idx])  # __eq__()
print('두 객체 리스트의 모든 데이터가 일치하는지 여부 :', all(result))

두 객체 리스트의 모든 데이터가 일치하는지 여부 : True


In [29]:
# json파일을 데이터프레임으로 읽기
import pandas as pd
member = pd.read_json('data/ch09_member2.json')  # encoding='utf-8' 기본
member

,name,age,email,address
0,홍길동,30,h@h.com,서울
1,김길동,25,k@h.com,부산
2,신길동,35,s@h.com,대전


In [30]:
# open()함수의 기본 encoding은 cp949, pd.read_~()함수의 기본 encoding은 utf-8

In [33]:
# member = pd.read_json('data/ch09_member1.json')  # 에러
member = pd.read_json('data/ch09_member1.json', encoding='cp949')
member

,name,age,email,address
0,홍길동,20,h@h.com,서울
1,김길동,30,k@h.com,부산
2,신길동,40,s@h.com,안양


In [34]:
type(member)

pandas.core.frame.DataFrame

# 5절. hdf5파일 쓰기/읽기(딕셔너리, 데이터프레임)
 - h5py 사용
## 5.1 hdf5파일 쓰기
1. 딕셔너리 데이터를 hdf5파일에 쓰기
2. 데이터프레임 데이터를 hdf5파일에 쓰기

In [35]:
import seaborn as sns
iris_df = sns.load_dataset('iris')
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [36]:
type(iris_df)

pandas.core.frame.DataFrame

In [37]:
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [39]:
iris_dic = iris_df.to_dict()  # 데이터프레임(2차원) 변수를 딕셔너리로 형변환
iris_dic

{'sepal_length': {0: 5.1,
  1: 4.9,
  2: 4.7,
  3: 4.6,
  4: 5.0,
  5: 5.4,
  6: 4.6,
  7: 5.0,
  8: 4.4,
  9: 4.9,
  10: 5.4,
  11: 4.8,
  12: 4.8,
  13: 4.3,
  14: 5.8,
  15: 5.7,
  16: 5.4,
  17: 5.1,
  18: 5.7,
  19: 5.1,
  20: 5.4,
  21: 5.1,
  22: 4.6,
  23: 5.1,
  24: 4.8,
  25: 5.0,
  26: 5.0,
  27: 5.2,
  28: 5.2,
  29: 4.7,
  30: 4.8,
  31: 5.4,
  32: 5.2,
  33: 5.5,
  34: 4.9,
  35: 5.0,
  36: 5.5,
  37: 4.9,
  38: 4.4,
  39: 5.1,
  40: 5.0,
  41: 4.5,
  42: 4.4,
  43: 5.0,
  44: 5.1,
  45: 4.8,
  46: 5.1,
  47: 4.6,
  48: 5.3,
  49: 5.0,
  50: 7.0,
  51: 6.4,
  52: 6.9,
  53: 5.5,
  54: 6.5,
  55: 5.7,
  56: 6.3,
  57: 4.9,
  58: 6.6,
  59: 5.2,
  60: 5.0,
  61: 5.9,
  62: 6.0,
  63: 6.1,
  64: 5.6,
  65: 6.7,
  66: 5.6,
  67: 5.8,
  68: 6.2,
  69: 5.6,
  70: 5.9,
  71: 6.1,
  72: 6.3,
  73: 6.1,
  74: 6.4,
  75: 6.6,
  76: 6.8,
  77: 6.7,
  78: 6.0,
  79: 5.7,
  80: 5.5,
  81: 5.5,
  82: 5.8,
  83: 6.0,
  84: 5.4,
  85: 6.0,
  86: 6.7,
  87: 6.3,
  88: 5.6,
  89: 5.5,
  90

In [40]:
list(iris_dic.keys())  # hdf파일 출력 시 group이 될 키

['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']

In [41]:
for group, value in iris_dic.items():
    print('그룹명 :', group)
    print(value)

그룹명 : sepal_length
{0: 5.1, 1: 4.9, 2: 4.7, 3: 4.6, 4: 5.0, 5: 5.4, 6: 4.6, 7: 5.0, 8: 4.4, 9: 4.9, 10: 5.4, 11: 4.8, 12: 4.8, 13: 4.3, 14: 5.8, 15: 5.7, 16: 5.4, 17: 5.1, 18: 5.7, 19: 5.1, 20: 5.4, 21: 5.1, 22: 4.6, 23: 5.1, 24: 4.8, 25: 5.0, 26: 5.0, 27: 5.2, 28: 5.2, 29: 4.7, 30: 4.8, 31: 5.4, 32: 5.2, 33: 5.5, 34: 4.9, 35: 5.0, 36: 5.5, 37: 4.9, 38: 4.4, 39: 5.1, 40: 5.0, 41: 4.5, 42: 4.4, 43: 5.0, 44: 5.1, 45: 4.8, 46: 5.1, 47: 4.6, 48: 5.3, 49: 5.0, 50: 7.0, 51: 6.4, 52: 6.9, 53: 5.5, 54: 6.5, 55: 5.7, 56: 6.3, 57: 4.9, 58: 6.6, 59: 5.2, 60: 5.0, 61: 5.9, 62: 6.0, 63: 6.1, 64: 5.6, 65: 6.7, 66: 5.6, 67: 5.8, 68: 6.2, 69: 5.6, 70: 5.9, 71: 6.1, 72: 6.3, 73: 6.1, 74: 6.4, 75: 6.6, 76: 6.8, 77: 6.7, 78: 6.0, 79: 5.7, 80: 5.5, 81: 5.5, 82: 5.8, 83: 6.0, 84: 5.4, 85: 6.0, 86: 6.7, 87: 6.3, 88: 5.6, 89: 5.5, 90: 5.5, 91: 6.1, 92: 5.8, 93: 5.0, 94: 5.6, 95: 5.7, 96: 5.7, 97: 6.2, 98: 5.1, 99: 5.7, 100: 6.3, 101: 5.8, 102: 7.1, 103: 6.3, 104: 6.5, 105: 7.6, 106: 4.9, 107: 7.3, 108: 6.7, 

In [45]:
for group, value in iris_dic.items():
    print('그룹 :', group)
    for key, data in value.items():
          print('{}:{}'.format(str(key), data), end='\t')
    print('\n')

그룹 : sepal_length
0:5.1	1:4.9	2:4.7	3:4.6	4:5.0	5:5.4	6:4.6	7:5.0	8:4.4	9:4.9	10:5.4	11:4.8	12:4.8	13:4.3	14:5.8	15:5.7	16:5.4	17:5.1	18:5.7	19:5.1	20:5.4	21:5.1	22:4.6	23:5.1	24:4.8	25:5.0	26:5.0	27:5.2	28:5.2	29:4.7	30:4.8	31:5.4	32:5.2	33:5.5	34:4.9	35:5.0	36:5.5	37:4.9	38:4.4	39:5.1	40:5.0	41:4.5	42:4.4	43:5.0	44:5.1	45:4.8	46:5.1	47:4.6	48:5.3	49:5.0	50:7.0	51:6.4	52:6.9	53:5.5	54:6.5	55:5.7	56:6.3	57:4.9	58:6.6	59:5.2	60:5.0	61:5.9	62:6.0	63:6.1	64:5.6	65:6.7	66:5.6	67:5.8	68:6.2	69:5.6	70:5.9	71:6.1	72:6.3	73:6.1	74:6.4	75:6.6	76:6.8	77:6.7	78:6.0	79:5.7	80:5.5	81:5.5	82:5.8	83:6.0	84:5.4	85:6.0	86:6.7	87:6.3	88:5.6	89:5.5	90:5.5	91:6.1	92:5.8	93:5.0	94:5.6	95:5.7	96:5.7	97:6.2	98:5.1	99:5.7	100:6.3	101:5.8	102:7.1	103:6.3	104:6.5	105:7.6	106:4.9	107:7.3	108:6.7	109:7.2	110:6.5	111:6.4	112:6.8	113:5.7	114:5.8	115:6.4	116:6.5	117:7.7	118:7.7	119:6.0	120:6.9	121:5.6	122:7.7	123:6.3	124:6.7	125:7.2	126:6.2	127:6.1	128:6.4	129:7.2	130:7.4	131:7.9	132:6.4	133:6.3	134:6.1	135:7.7	136:

In [46]:
# 딕셔너리 데이터를 hdf파일로 쓰기
import h5py
with h5py.File('data/ch09_iris.hdf5', 'w') as f:
    for group, value in iris_dic.items():
        grp = f.create_group(group)
        for key, data in value.items():
              grp.create_dataset(str(key), data=data)

In [48]:
# 데이터프레임을 hdf파일로 쓰기
iris_df.to_hdf('data/ch09_iris2.hdf5', key='iris')

In [49]:
iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


## 5.2 hdf5파일 읽기
1. hdf5파일에서 데이터프레임으로 읽기
2. hdf5파일에서 딕셔너리로 읽기

In [50]:
# hdf5파일에서 데이터프레임 변수로 읽어오기
load_iris_df = pd.read_hdf('data/ch09_iris2.hdf5', key='iris')
load_iris_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [51]:
# hdf5파일을 딕셔너리 변수로 읽어오기

In [52]:
# 딕셔너리에 넣을 데이터를 가져와서 딕셔너리에 추가
dict_data = {}
dict_data['name'] = '홍길동'
dict_data

{'name': '홍길동'}

In [53]:
# 위와 같은 출력
dict_data = {}
dict_data.update({'name' : '홍길동'})
dict_data

{'name': '홍길동'}

In [64]:
# hdf5파일의 내용을 읽어 출력하기
with h5py.File('data/ch09_iris.hdf5', 'r') as f:
    for group, value in f.items():
        print('그룹 :', group)
        # print(value)
        for key, data in value.items():
            print('{}:{}'.format(key, data[()]), end='\t')  # data[()] 유무 차이 확인
        print()

그룹 : petal_length
0:1.4	1:1.4	10:1.5	100:6.0	101:5.1	102:5.9	103:5.6	104:5.8	105:6.6	106:4.5	107:6.3	108:5.8	109:6.1	11:1.6	110:5.1	111:5.3	112:5.5	113:5.0	114:5.1	115:5.3	116:5.5	117:6.7	118:6.9	119:5.0	12:1.4	120:5.7	121:4.9	122:6.7	123:4.9	124:5.7	125:6.0	126:4.8	127:4.9	128:5.6	129:5.8	13:1.1	130:6.1	131:6.4	132:5.6	133:5.1	134:5.6	135:6.1	136:5.6	137:5.5	138:4.8	139:5.4	14:1.2	140:5.6	141:5.1	142:5.1	143:5.9	144:5.7	145:5.2	146:5.0	147:5.2	148:5.4	149:5.1	15:1.5	16:1.3	17:1.4	18:1.7	19:1.5	2:1.3	20:1.7	21:1.5	22:1.0	23:1.7	24:1.9	25:1.6	26:1.6	27:1.5	28:1.4	29:1.6	3:1.5	30:1.6	31:1.5	32:1.5	33:1.4	34:1.5	35:1.2	36:1.3	37:1.4	38:1.3	39:1.5	4:1.4	40:1.3	41:1.3	42:1.3	43:1.6	44:1.9	45:1.4	46:1.6	47:1.4	48:1.5	49:1.4	5:1.7	50:4.7	51:4.5	52:4.9	53:4.0	54:4.6	55:4.5	56:4.7	57:3.3	58:4.6	59:3.9	6:1.4	60:3.5	61:4.2	62:4.0	63:4.7	64:3.6	65:4.4	66:4.5	67:4.1	68:4.5	69:3.9	7:1.5	70:4.8	71:4.0	72:4.9	73:4.7	74:4.3	75:4.4	76:4.8	77:5.0	78:4.5	79:3.5	8:1.4	80:3.8	81:3.7	82:3.9	83:5.1	84:4.5	85:

In [60]:
# hdf5파일의 내용을 딕셔너리에 담기
iris_dic2 = {}
with h5py.File('data/ch09_iris.hdf5', 'r') as f:
    for group, value in f.items():
        cols ={}
        for key, data in value.items():
            # cols[int(key)] = data[()]  # cols[0] = 1.3
            cols.update({int(key):data[()]})
        # iris_dic2[group] = cols
        iris_dic2.update({group : cols})

In [63]:
for group, value in iris_dic2.items():
    print('그룹 :', group)
    print(value)
    print()  # 개행

그룹 : petal_length
{0: 1.4, 1: 1.4, 10: 1.5, 100: 6.0, 101: 5.1, 102: 5.9, 103: 5.6, 104: 5.8, 105: 6.6, 106: 4.5, 107: 6.3, 108: 5.8, 109: 6.1, 11: 1.6, 110: 5.1, 111: 5.3, 112: 5.5, 113: 5.0, 114: 5.1, 115: 5.3, 116: 5.5, 117: 6.7, 118: 6.9, 119: 5.0, 12: 1.4, 120: 5.7, 121: 4.9, 122: 6.7, 123: 4.9, 124: 5.7, 125: 6.0, 126: 4.8, 127: 4.9, 128: 5.6, 129: 5.8, 13: 1.1, 130: 6.1, 131: 6.4, 132: 5.6, 133: 5.1, 134: 5.6, 135: 6.1, 136: 5.6, 137: 5.5, 138: 4.8, 139: 5.4, 14: 1.2, 140: 5.6, 141: 5.1, 142: 5.1, 143: 5.9, 144: 5.7, 145: 5.2, 146: 5.0, 147: 5.2, 148: 5.4, 149: 5.1, 15: 1.5, 16: 1.3, 17: 1.4, 18: 1.7, 19: 1.5, 2: 1.3, 20: 1.7, 21: 1.5, 22: 1.0, 23: 1.7, 24: 1.9, 25: 1.6, 26: 1.6, 27: 1.5, 28: 1.4, 29: 1.6, 3: 1.5, 30: 1.6, 31: 1.5, 32: 1.5, 33: 1.4, 34: 1.5, 35: 1.2, 36: 1.3, 37: 1.4, 38: 1.3, 39: 1.5, 4: 1.4, 40: 1.3, 41: 1.3, 42: 1.3, 43: 1.6, 44: 1.9, 45: 1.4, 46: 1.6, 47: 1.4, 48: 1.5, 49: 1.4, 5: 1.7, 50: 4.7, 51: 4.5, 52: 4.9, 53: 4.0, 54: 4.6, 55: 4.5, 56: 4.7, 57: 3.3, 5

# 6절. 연습문제 : 고객관리 프로그램
 - 프로그램 실행 시작 시 : 
     * 'data/ch09_customers.txt'(프로그램 실행 시 백업용 데이터)파일이 있으면 파일 input
     * 'data/ch09_customers.txt' 파일이 없으면 빈 파일 생성
     * 'data/ch09_customers.txt' 파일 내용 : 
     ```
         홍길동, 010-9999-9999, hong@m.com, 30, 5, 열심히
         홍길동, 010-8888-8888, hong2@m.com, 31, 5, abc
         유길동, 010-5432-2222, yu@hong.com, 20, 2, 가칠해
     ```
 - Customer 클래스 작성(고객데이터 타입) Customer('홍길동', '010', 'h@h', 30, 5, '기타')
     * __init__, __str__, as_dic(csv파일 출력을 위해서), to_list_style(txt파일 출력을 위해 list로 return)
 - 1입력
 

In [526]:
class Customer:
    '고객 데이터 저장'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def as_dic(self):  # 객체를 딕셔너리데이터로 반환 (csv파일 저장시)
        return {'name' : self.name,
                'phone' : self.phone,
                'email' : self.email,
                'age' : self.age,
                'grade' : self.grade,
                'etc' : self.etc}
    def to_list_style(self):   # 객체를 list return([홍길동, 010-8999-9999, e@e.com, 20, 3, 까칠해])
        temp = [self.name, self.phone, self.email, str(self.age), str(self.grade), self.etc]
        return ', '.join(temp)
    def __str__(self):  # *** 홍길동 010-8999-9999 e@e.com 20 까칠해
        return  '{:>5}\t{:3}\t{:15}\t{:15}\t{:3}\t{}'.format(  # '*' * self.grade
                                self.grade * ('*') ,self.name, self.phone, self.email, self.age, self.etc)

In [527]:
# Customer class test
u = Customer('홍', '010-999-9999', 'h@h', 20, 3, '까칠해')
u1 = Customer('홍', '010-999-9999', 'h@h', 20, 5, '까칠해')
print(u)
print(u1)

  ***	홍  	010-999-9999   	h@h            	 20	까칠해
*****	홍  	010-999-9999   	h@h            	 20	까칠해


In [573]:
c1 = Customer("홍길동","010-9999-9999","h@h.com", 20, 3, "기타")
c2 = Customer("홍길동","010-9999-9999","h@h.com", 20, 5, "기타")
print(c1)
print(c2)
print(c1.to_list_style())
print(c1.as_dic())

  ***	홍길동	010-9999-9999  	h@h.com        	 20	기타
*****	홍길동	010-9999-9999  	h@h.com        	 20	기타
홍길동, 010-9999-9999, h@h.com, 20, 3, 기타
{'name': '홍길동', 'phone': '010-9999-9999', 'email': 'h@h.com', 'age': 20, 'grade': 3, 'etc': '기타'}


In [528]:
def to_customer(line):  # txt파일 내용 한줄(홍길동, 010-8999-9999, e@e.com, 20, 3, 까칠해)을 Customer 객체로 반환
    data = line.strip().split(',')
    name = data[0]
    phone = data[1].strip()
    email = data[2].strip()
    age = int(data[3].strip())
    grade = int(data[4].strip())
    etc = data[5].strip()
    return Customer(name, phone, email, age, grade, etc)

## 0. 파일 내용 가져오기
- 실행하면 data/ch09_customers.txt 파일의 내용을 load(customer_list)
- data/ch09_customers.txt이 존재하지 않으면
- 빈 data/ch09_customers.txt파일을 생성하고
- 데이터는 customer_list=[]

In [529]:
def load_customers():
    customer_list = []
    try:
        with open('data/ch09_customers.txt', 'r', encoding='utf-8') as f:
            # 방법 1. 한 줄씩 읽어서 customer_list에 append하기
            # lines = f.readline()
            # while lines != '':
            #     customer = to_customer(line)
            #     customer_list.append(customer)
            #     line = f.readline()
            # 방법 2. 한 번에 읽어서 for문으로 append하기
            lines = f.readlines()
        for line in lines:
            customer = to_customer(line)
            print(customer)
            customer_list.append(customer)
    except FileNotFoundError as e:
        print('데이터 파일 ch09_customers.txt가 없어서 초기화합니다.')
        with open('data/ch09_customers.txt', 'w', encoding='utf-8') as f:
            f.write('')
    return customer_list

In [574]:
# 
def load_customers():
    customer_list = []
    # 파일의 내용을 customer_list에 append
    # 파일이 없으면 빈 파일 생성하고 내용은 ''추가
    try:
        with open('data/ch09_customers.txt', 'r', encoding='utf-8') as f:
            # 방법1. 한줄씩 읽어서 customer_list에 append하기
#             line = f.readline()
#             while line != '':
#                 customer = to_customer(line)
#                 customer_list.append(customer)
            # 방법2. 한꺼번에 읽어서 for문으로 append하기
            lines = f.readlines()
        for line in lines:
            customer = to_customer(line)
            customer_list.append(customer)
    except FileNotFoundError as e:
        print('데이터 파일 ch09_customers.txt가 없어서 초기화합니다')
        with open('data/ch09_customers.txt', 'w', encoding='utf-8') as f:
            f.write('')
    return customer_list

In [530]:
# 위 함수 test - 아래 부분 test를 위해 customer_list에 할당
customer_list = load_customers()
customer_list

*****	홍길동	010-9999-9999  	hong@m.com     	 30	열심히
*****	홍길동	010-8888-8888  	hong2@m.com    	 31	abc
   **	유길동	010-5432-2222  	yu@hong.com    	 20	가칠해


In [575]:
# 위 함수 test - 아래 부분 test를 위해 customer_list에 할당
customer_list = load_customers()
for customer in customer_list:
    print(customer)

*****	홍길동	010-9999-9999  	hong@m.com     	 30	열심히
*****	홍길동	010-8888-8888  	hong2@m.com    	 31	abc
   **	유길동	010-5432-2222  	yu@hong.com    	 20	가칠해
   **	홍  	010            	h@             	 20	ㅇ


## 1. 입력

In [570]:
# 1. 입력
def fn1_insert_customer_info():
    name = input('이름을 입력하세요 :')
    phone = input('전화번호를 입력하세요 :')
    email = input('메일을 입력하세요 :')
    age = int(input('나이를 입력하세요'))
    grade = int(input('GRADE를 입력하세요 :'))
    etc = input('기타 내용을 입력하세요 :')
    return Customer(name, phone, email, age, grade, etc) 

In [2]:
## 위 함수 test
# fn1_insert_customer_info()

In [576]:
#
def fn1_insert_customer_info():
    '사용자로부터 이름, 전화번호, 이메일, 나이, 등급, etc를 받아 customer형 객체 return'
    name = input('이름 : ')
    phone = input("전화번호 : ")
    email = input("이메일 : ")
    try:
        age = int(input("나이 : "))
        if age<0:
            age = 0
    except ValueError as e:
        print('유효하지 않은 나이  입력시 나이는 0으로 초기화')
        age = 0
    try:
        grade = int(input("고객등급(1~5) : "))
        if grade < 1 :
            grade = 1
        if grade > 5 :
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력시 등급은 1로 초기화')
        grade = 1
    etc = input("기타 정보 : ")
    customer = Customer(name, phone, email, age, grade, etc)
    return customer

In [577]:
# 위 함수 테스트
customer = fn1_insert_customer_info()
print('입력한 정보로 만든 Member 객체 :',customer)
customer_list.append(customer)

이름 : ㅇ
전화번호 : ㅇ
이메일 : ㅇ
나이 : ㅇ
유효하지 않은 나이  입력시 나이는 0으로 초기화
고객등급(1~5) : ㅇ
유효하지 않은 등급을 입력시 등급은 1로 초기화
기타 정보 : ㅇ
입력한 정보로 만든 Member 객체 :     *	ㅇ  	ㅇ              	ㅇ              	  0	ㅇ


## 2. 전체 출력

In [286]:
# 2. 전체 출력
def fn2_print_customers(customer_list):
    'customer_list를 전체 출력(*** 홍길동 010-8999-9999 e@e.com 20 까칠해)'
    print('==============================================================')
    print("\t\t\t고객 정보")
    print('--------------------------------------------------------------')
    print('GRADE\t이름\t전화\t\t메일\t나이\t기타\t')
    print('==============================================================')
    for customer in customer_list:
        print(customer)
    print('==============================================================')

In [287]:
# 위 함수 test
fn2_print_customers(customer_list)

			고객 정보
--------------------------------------------------------------
GRADE	이름	전화		메일	나이	기타	
*****	홍길동	010-9999-9999  	hong@m.com     	 30	열심히
*****	홍길동	010-8888-8888  	hong2@m.com    	 31	abc
   **	유길동	010-5432-2222  	yu@hong.com    	 20	가칠해


In [578]:
def fn2_print_customers(customer_list):
    'customer_list를 전체 출력(*** 홍길동 010-8999-9999 e@e.com 20 까칠해)'
    print('='*70)
    print("{:^70}".format("고객 정보"))
    print('-'*70)
    print("{:^5}\t{:^3}\t{:^10}{:^15}\t{:^3}\t{}".format("GRADE",
                    "이름", "전화", "메일", "나이", "기타"))
    print('-'*70)
    for customer in customer_list:
        print(customer)

In [579]:
# 위 함수 테스트
fn2_print_customers(customer_list)

                                고객 정보                                 
----------------------------------------------------------------------
GRADE	이름 	    전화          메일       	나이 	기타
----------------------------------------------------------------------
*****	홍길동	010-9999-9999  	hong@m.com     	 30	열심히
*****	홍길동	010-8888-8888  	hong2@m.com    	 31	abc
   **	유길동	010-5432-2222  	yu@hong.com    	 20	가칠해
   **	홍  	010            	h@             	 20	ㅇ
    *	ㅇ  	ㅇ              	ㅇ              	  0	ㅇ


## 3. 삭제

In [427]:
# 3. 삭제 (동명이인이 있을 때)
def fn3_delete_customer(customer_list):
    '''
    삭제하고자 하는 고객 이름을 받아 매개변수로 들어온 customer_list에서 삭제
    삭제여부를 출력하면 더 좋음
    '''
    name = input('삭제할 고객의 이름은? ')
    deleted_flag = False  # 삭제했는지 여부 저장
    for customer in customer_list:
        if customer.name == name:
            customer_list.remove(customer)
            deleted_flag = True
    if deleted_flag:
        print('검색하신 {}님의 고객 정보를 삭제하였습니다.'.format(name))
    else:
        print('검색하신 {}님의 고객 정보가 존재하지 않습니다.'.format(name))

In [524]:
# 3. 삭제 (동명이인이 있을 때)
def fn3_delete_customer(customer_list):
    '''
    삭제하고자 하는 고객 이름을 받아 매개변수로 들어온 customer_list에서 삭제
    삭제여부를 출력하면 더 좋음
    '''
    name = input('삭제할 고객의 이름은? ')
    deleted_flag = False  # 삭제했는지 여부 저장
    for customer in customer_list:
        if customer.name == name:
            customer_list.remove(customer)
            deleted_flag = True
        print('검색하신 {}님의 고객 정보를 삭제하였습니다.'.format(name))
    else:
        print('검색하신 {}님의 고객 정보가 존재하지 않습니다.'.format(name))

In [559]:
# 3. 삭제 (동명이인이 있을 때)
def fn3_delete_customer(customer_list):
    '''
    삭제하고자 하는 고객 이름을 받아 매개변수로 들어온 customer_list에서 삭제
    삭제여부를 출력하면 더 좋음
    '''
    name = input('삭제할 고객의 이름은? ')
    deleted_flag = False  # 삭제했는지 여부 저장
    for customer in customer_list:
        if customer.name == name:
            customer_list.remove(customer)
            deleted_flag = True
        print('검색하신 {}님의 고객 정보를 삭제하였습니다.'.format(name))
    else:
        print('검색하신 {}님의 고객 정보가 존재하지 않습니다.'.format(name))

## 3. 삭제 (동명이인이 없다는 전제에서의 구현)

In [580]:
def fn3_delete_customer(customer_list):
    '''
    삭제하고자 하는 고객 이름을 받아 매개변수로 들어온 customer_list에서 삭제
    삭제여부를 출력하면 더 좋음
    '''
    name = input('삭제할 고객의 이름은?')
    delete_flag = False # 삭제했는지 여부 저장
    for idx, customer in enumerate(customer_list):
        if customer.name == name:
            customer_list.remove(customer)
            # del customer_list[idx]
            delete_flag = True
            break
    if delete_flag:
        print(name, '님을 삭제하였습니다')
    else:
        print(name, '님이 데이터에 존재하지 않습니다')

In [3]:
## 위 함수 test
# fn3_delete_customer(customer_list)

## 3. 삭제 (동명이인이 있다는 전제에서의 구현)

In [582]:
# 3. 삭제 (동명이인이 있다는 전제에서의 구현)
def fn3_delete_customer(customer_list):
    '''
    삭제하고자 하는 고객 이름을 받아 매개변수로 들어온 customer_list에서 삭제
    삭제여부를 출력하면 더 좋음
    '''
    name = input('삭제할 고객의 이름은?')
    delete_flag = False # 삭제했는지 여부 저장
    delete_idx = []
    for idx, customer in enumerate(customer_list):
        if customer.name == name:
            delete_flag = True
            delete_idx.append(idx)
            # 여기서 idx번째를 지우면 idx이후가 앞으로 밀려내려옴
    # 동명이인일 경우 끝idx부터 지워야 제대로 작동
    for i in range(len(delete_idx)-1, -1, -1):
        del customer_list[delete_idx[i]]
    if delete_flag:
        print(name, '님', len(delete_idx), '명을 삭제하였습니다')
    else:
        print(name, '님이 데이터에 존재하지 않습니다')

In [581]:
# 위 함수 테스트
fn3_delete_customer(customer_list)

삭제할 고객의 이름은?ㅇ
ㅇ 님을 삭제하였습니다


## 4.  이름찾기

In [334]:
# 4. 이름찾기 (동명이인이 있다고 할지 없다고 할지 정한대로 구현)
def fn4_search_customer(customer_list):
    '찾고자 하는 고객이름을 받아 찾는 고객 정보를 출력'
    name = input('검색할 고객의 이름은? ')
    search_result = False
    for customer in customer_list:
        if customer.name == name:
            search_result = True
            print(customer)
    if search_result:
        pass
    else:
        print('검색하신 {}님의 고객 데이터가 존재하지 않습니다.'.format(name))

In [566]:
# 4. 이름찾기 (동명이인 있음)
def fn4_search_customer(customer_list):
    name = input('검색할 고객이름 : ')
    name_= False
    for customer in customer_list:
        if customer.name == name :
            name_ = True
            print(customer)
    if name_ :
        pass
    else :
        print('검색하신 {} 고객 정보가 없습니다'.format(name))

In [565]:
# 위 함수 test
fn4_search_customer(customer_list)

검색할 고객이름 :홍길동
*****	홍길동	010-9999-9999  	hong@m.com     	 30	열심히
*****	홍길동	010-8888-8888  	hong2@m.com    	 31	abc


In [584]:
# 
def fn4_search_customer(customer_list):
    '찾고자 하는 고객이름을 받아 찾는 고객 정보를 출력'
    name = input('검색할 고객의 이름은?')
    search_result = []
    for customer in customer_list:
        if customer.name == name :
            search_result.append(customer)
    if len(search_result):
        print('='*70)
        print("{:^70}".format(name+"님 고객 정보"))
        print('-'*70)
        print("{:^5}\t{:^3}\t{:^10}{:^15}\t{:^3}\t{}".format("GRADE",
                        "이름", "전화", "메일", "나이", "기타"))
        print('-'*70)
        for customer in search_result:
            print(customer)
    else:
        print('검색한 고객이 데이터에 존재하지 않습니다')

In [585]:
# 위 함수 테스트
fn4_search_customer(customer_list)

검색할 고객의 이름은?ㅇ
검색한 고객이 데이터에 존재하지 않습니다


In [586]:
list(customer_list[0].__dict__.keys()) # csv 해더로 들어갈 예정

['name', 'phone', 'email', 'age', 'grade', 'etc']

## 5. 내보내기(CSV)

In [262]:
# 5. 내보내기(CSV)
def fn5_save_customer_csv(customer_list):
    with open('data/ch09_customers.csv', 'w', encoding='utf-8', newline='') as f:
        dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
        for customer in customer_list:
            dict_writer.writerow(customer.as_dic())

In [587]:
#
def fn5_save_customer_csv(customer_list):
    '저장할 csv파일명을 입력받아 data폴더 밑에 customer_list의 내용을 csv파일로 출력(저장)'
    import csv
    customer_dict_list = []
    for customer in customer_list:
        customer_list_dict.append(customer.as_dic())
    fieldnames = list(customer_list[0].__dict__.keys())
    filename = input('저장할 CSV 파일 이름은 ?')
    try :
        with open('data/'+filename, 'w', newline='', encoding='UTF-8') as f:
            dict_writer = csv.DictWriter(f, fieldnames=fieldnames)
            dict_writer.writeheader()
            dict_writer.writerows(customer_dict_list)
    except Exception as e:
        print('데이터 내보내기 예외 :', e)

In [4]:
## 위 함수 테스트
# fn5_save_customer_csv(customer_list)

## 9. 종료

In [567]:
# 9. 종료 (종료하기 전 customer_list를 txt파일에 저장하고 종료)
def fn9_save_customer_txt(customer_list):
    lst = []
    for customer in customer_list:
        lst.append(customer.to_list_style() + '\n')
    with open('data/ch09_customers.txt', 'w', encoding='utf-8') as f:
        f.writelines(lst)

In [589]:
# 위 함수 테스트
fn9_save_customer_txt(customer_list)

In [572]:
def main():
    global customer_list
    customer_list = load_customers()  # ch09_customers.txt의 내용을 load
    while True:
        print("1:입력","2:전체출력","3:삭제","4:이름찾기","5:내보내기(CSV)", "9:종료", sep=' | ', end=' ')
        fn = int(input('메뉴선택 : '))
        if fn == 1:
            customer = fn1_insert_customer_info()  # 입력받은 내용으로 customer객체를 반환
            customer_list.append(customer)
        elif fn == 2:
            fn2_print_customers(customer_list)
        elif fn == 3:
            fn3_delete_customer(customer_list)
        elif fn == 4:
            fn4_search_customer(customer_list)
        elif fn == 5:
            fn5_save_customer_csv(customer_list)
        elif fn == 9:
            fn9_save_customer_txt(customer_list)
            break
            
if __name__ == '__main__':
        main()

*****	홍길동	010-9999-9999  	hong@m.com     	 30	열심히
*****	홍길동	010-8888-8888  	hong2@m.com    	 31	abc
   **	유길동	010-5432-2222  	yu@hong.com    	 20	가칠해
1:입력 | 2:전체출력 | 3:삭제 | 4:이름찾기 | 5:내보내기(CSV) | 9:종료 메뉴선택 : 1
이름을 입력하세요 :홍
전화번호를 입력하세요 :010
메일을 입력하세요 :h@
나이를 입력하세요20
GRADE를 입력하세요 :2
기타 내용을 입력하세요 :ㅇ
1:입력 | 2:전체출력 | 3:삭제 | 4:이름찾기 | 5:내보내기(CSV) | 9:종료 메뉴선택 : 2
			고객 정보
--------------------------------------------------------------
GRADE	이름	전화		메일	나이	기타	
*****	홍길동	010-9999-9999  	hong@m.com     	 30	열심히
*****	홍길동	010-8888-8888  	hong2@m.com    	 31	abc
   **	유길동	010-5432-2222  	yu@hong.com    	 20	가칠해
   **	홍  	010            	h@             	 20	ㅇ
1:입력 | 2:전체출력 | 3:삭제 | 4:이름찾기 | 5:내보내기(CSV) | 9:종료 메뉴선택 : 9


In [205]:
# 1.다음중 Open 함수에 대해 잘못 설명한 것은? 3
# ① 함수가 반환하는 것은 열린 파일 객체이다.
# ② 파일을 열때 사용하는 모드는 읽기(r), 쓰기(w), 추가(a), 바이너리(b) 모드가 있다.
# ③ 바이너리(b) 모드와 쓰기(w) 모드는 같이 사용할 수 없다. : 피클링에서만
# ④ 파일의 인코딩은 encoding 속성을 이용해 설정할 수 있다.

In [208]:
# 2.피클링(pickling)에 대해 잘못 설명한 것은? 4
# ① 파이썬의 객체직렬화(Object Serialization) 방법이다.
# ② pickle 모듈의 dump()와 load() 함수를 이용해 객체를 쓰고 읽는다.
# ③ 파이썬 객체를 별도의 텍스트 변환 과정없이 파일에 직접 쓰고 읽는 것을 의미한다.
# ④ 피클링하기 위한 모드는 쓰기모드('w') 여야 한다. : wb

In [207]:
# 3.다음중 파이썬의 입/출력 모듈에 대한 설명 중 잘못된 것은? 1
# ① Pandas 패키지를 이용하면 리스트 데이터를 쉽게 파일에 쓰고 읽을수 있다. : 데이터프레임
# ② json 모듈의 dump(), load() 함수를 이용해 JSON 데이터를 파일에 읽고 쓸 수 있다.
# ③ h5py 모듈의 File 클래스를 이용해 HDF5 데이터를 파일에 읽고 쓸 수 있다.
# ④ csv 모듈의 reader(), writer() 함수를 이용해 CSV 파일을 읽고 쓸 수 있다.